# Pandas

This module assumes you've had some previous exposure to Pandas (so, we're not going into detail about what a dataframe is, for instance), but it's all right if you don't remember all the details. 

First, we'll import Pandas in the traditional way:

In [1]:
import pandas as pd

In [2]:
pd.show_versions()



INSTALLED VERSIONS
------------------
commit           : db08276bc116c438d3fdee492026f8223584c477
python           : 3.8.5.final.0
python-bits      : 64
OS               : Windows
OS-release       : 10
Version          : 10.0.17134
machine          : AMD64
processor        : AMD64 Family 23 Model 113 Stepping 0, AuthenticAMD
byteorder        : little
LC_ALL           : None
LANG             : None
LOCALE           : English_United States.1252

pandas           : 1.1.3
numpy            : 1.19.2
pytz             : 2020.1
dateutil         : 2.8.1
pip              : 20.2.4
setuptools       : 50.3.1.post20201107
Cython           : 0.29.21
pytest           : 6.1.1
hypothesis       : None
sphinx           : 3.2.1
blosc            : None
feather          : None
xlsxwriter       : 1.3.7
lxml.etree       : 4.6.1
html5lib         : 1.1
pymysql          : None
psycopg2         : None
jinja2           : 2.11.2
IPython          : 7.19.0
pandas_datareader: None
bs4              : 4.9.3
bottleneck   

Next, we'll read a CSV into a dataframe. 

In this case, we're using the "lifetime" dataset of Allegheny County dog licenses from the [Western PA Regional Data Center](https://data.wprdc.org/dataset/allegheny-county-dog-licenses). The only change I've made to this file, prior to importing it, was changing the filename to "dog_licenses.csv." Otherwise, the data, column headers, etc. are all the same as the file at WPRDC.

In [3]:
# you have to love how easy it is to pull a CSV into a dataframe
# (you make your life MUCH easier if you deal with datetimes on input)
licenses = pd.read_csv("dog_licenses.csv", parse_dates=["ValidDate"])

# show the beginning of the file
licenses.head()

,LicenseType,Breed,Color,DogName,OwnerZip,ExpYear,ValidDate
0,Dog Lifetime Neutered Male,POODLE MIN,WHITE/BLACK/GREY,BAILEY,15137,2099,2005-08-08
1,Dog Lifetime Neutered Male,AM PIT BULL TERRIER,BRINDLE,SPENCER,15045,2099,2006-03-07
2,Dog Lifetime Spayed Female,DALMATIAN,SPOTTED,CLEO,15110,2099,2005-08-08
3,Dog Senior Lifetime Neutered Male,COCKER SPANIEL,BUFF,HARLEY,15236,2099,2005-08-08
4,Dog Lifetime Spayed Female,CHIHUAHUA,WHITE/BROWN,ANGEL,15216,2099,2005-08-08


Pandas will tell us the facts we need to know about our data.

In [4]:
print("Shape, (rows, columns):", licenses.shape, "\n")
print("Size:", licenses.size, "\n")
print("Data type of items:", licenses.dtypes, "\n")

Shape, (rows, columns): (33156, 7) 

Size: 232092 

Data type of items: LicenseType            object
Breed                  object
Color                  object
DogName                object
OwnerZip                int64
ExpYear                 int64
ValidDate      datetime64[ns]
dtype: object 



In [5]:
# and we can get descriptive statistics, of a sort,
# for the two columns it considers "numeric"
licenses.describe()

,OwnerZip,ExpYear
count,33156.000000,33156.0
mean,15156.003197,2099.0
std,298.770228,0.0
min,15003.000000,2099.0
25%,15102.000000,2099.0
50%,15137.000000,2099.0
75%,15228.000000,2099.0
max,47909.000000,2099.0


It's good to know about `.describe()`, but honestly, neither of those columns should be described statistically. One's a zip code (a categorical variable, probably best handled as a string), and the other's a year, which is probably OK as an int, provided we don't do silly things like summing or averaging it.

### Changing the data type of a column

Actually, it's really straightforward! (Unless you're changing not-a-datetime to a datetime, which is part of why I try to parse them as I pull them in.)

Tell Pandas which columns you want to change, and use `.astype("type")` to set each one.

In [7]:
# change data type of zip code field to string
licenses["OwnerZip"] = licenses["OwnerZip"].astype("string")

# and, actually, we should really make our other strings into string types, too
cols_list = ["LicenseType", "Breed", "Color", "DogName"]
licenses[cols_list] = licenses[cols_list].astype("string")

## Using `.groupby()`

`.groupby()` gives us the ability to look at our data in subsets and run functions on combinations of our data. 

The syntax is
```Python
smaller_df = primary_df.groupby("column_to_be_grouped")["column_to_operate_on"].function()
```

### grouping by a single column

Let's count how many dogs are in each zip code. In this case, the "column_to_operate_on" is literally any column, since we're just counting.

In [8]:
dogs_by_zip = licenses.groupby("OwnerZip")["DogName"].count()
dogs_by_zip.head()

OwnerZip
15003    28
15005    43
15006     6
15007     9
15012     2
Name: DogName, dtype: int64

In [9]:
# hmm
type(dogs_by_zip)

pandas.core.series.Series

Often you'd really rather have a dataframe to work with than a series. If so, you can use `as_index=False` to make your groups back into a column (or, as you'll see, columns) in their own right.

In [10]:
dogs_by_zip = licenses.groupby("OwnerZip", as_index=False)["DogName"].count()
dogs_by_zip.head()

,OwnerZip,DogName
0,15003,28
1,15005,43
2,15006,6
3,15007,9
4,15012,2


#### Renaming our columns

Unsurprisingly, we use `.rename()`, but perhaps-surprisingly, it takes a dictionary as its input:
```Python
df = df.rename(columns = {"old_column_name1" : "new_column_name1", "ocn2" : "ncn2", ...})
```

The column where we keep the counts probably shouldn't be called "DogName" anymore.

In [11]:
dogs_by_zip = dogs_by_zip.rename(columns = {"DogName" : "Count"})
dogs_by_zip.head()

,OwnerZip,Count
0,15003,28
1,15005,43
2,15006,6
3,15007,9
4,15012,2


In [12]:
# OK, how many are there in Downtown (15219)?
dogs_by_zip[dogs_by_zip.OwnerZip == "15219"]

,OwnerZip,Count
96,15219,12


### grouping by multiple columns

You need to pass in your list of columns as, well, a list. 

In the example below, we count how many dogs there are by zip code and by breed. So, then, if we want to know how many poodles are in a particular zip code, we can do that.

In [13]:
zab_df = licenses.groupby(["OwnerZip", "Breed"], as_index=False)["DogName"].count()
zab_df = zab_df.rename(columns = {"DogName" : "Count"})
zab_df.head()

,OwnerZip,Breed,Count
0,15003,AM PIT BULL TERRIER,1
1,15003,AM PITT BULL MIX,1
2,15003,AUS SHEPHERD,1
3,15003,BICHON FRISE,1
4,15003,ENG SPRINGER SPANIE,3


#### A good trick: `.str.contains()`

We can look for substrings within our columns if we want to. Say you want to look for all pitt bulls (and pitbulls), including mixes with other breeds. Not a problem. 

In [14]:
# how many pitbulls live near CCAC North?
pitties = zab_df[zab_df['Breed'].str.contains('PIT')]
pitties_by_north_campus = pitties[pitties.OwnerZip == "15237"]

print("Total pitties by North Campus:", sum(pitties_by_north_campus.Count), "\n")
print(pitties_by_north_campus) # there's a reason I don't always explicitly print dfs

Total pitties by North Campus: 42 

     OwnerZip                Breed  Count
5947    15237  AM PIT BULL TERRIER     21
5948    15237     AM PITT BULL MIX     21


In [15]:
# I did say "poodles," though
oodles = zab_df[zab_df["Breed"].str.contains("OODLE")]
CCAC_North_area_poodle_mixes = oodles[oodles.OwnerZip == "15237"]
CCAC_North_area_poodle_mixes

,OwnerZip,Breed,Count
6015,15237,GOLDENDOODLE,55
6029,15237,LABRADOODLE,22
6053,15237,POODLE MED,1
6054,15237,POODLE MIN,2
6055,15237,POODLE MIX,17
6056,15237,POODLE STANDARD,20
6057,15237,POODLE TOY,3
6071,15237,SCHNOODLE,6


In [16]:
sum(CCAC_North_area_poodle_mixes.Count)

126

## Reorganizing columns

Surprisingly easy, actually. Say you have a dataframe with three columns, "b", "c", and "a", and they're in that order, which is not the order you want.
```Python
df = df[["a", "b", "c"]]
```


In [17]:
licenses.head()

,LicenseType,Breed,Color,DogName,OwnerZip,ExpYear,ValidDate
0,Dog Lifetime Neutered Male,POODLE MIN,WHITE/BLACK/GREY,BAILEY,15137,2099,2005-08-08
1,Dog Lifetime Neutered Male,AM PIT BULL TERRIER,BRINDLE,SPENCER,15045,2099,2006-03-07
2,Dog Lifetime Spayed Female,DALMATIAN,SPOTTED,CLEO,15110,2099,2005-08-08
3,Dog Senior Lifetime Neutered Male,COCKER SPANIEL,BUFF,HARLEY,15236,2099,2005-08-08
4,Dog Lifetime Spayed Female,CHIHUAHUA,WHITE/BROWN,ANGEL,15216,2099,2005-08-08


In [18]:
licenses2 = licenses[["OwnerZip", "DogName", "Breed", "Color", "LicenseType", "ExpYear", "ValidDate"]]
licenses2.head()

,OwnerZip,DogName,Breed,Color,LicenseType,ExpYear,ValidDate
0,15137,BAILEY,POODLE MIN,WHITE/BLACK/GREY,Dog Lifetime Neutered Male,2099,2005-08-08
1,15045,SPENCER,AM PIT BULL TERRIER,BRINDLE,Dog Lifetime Neutered Male,2099,2006-03-07
2,15110,CLEO,DALMATIAN,SPOTTED,Dog Lifetime Spayed Female,2099,2005-08-08
3,15236,HARLEY,COCKER SPANIEL,BUFF,Dog Senior Lifetime Neutered Male,2099,2005-08-08
4,15216,ANGEL,CHIHUAHUA,WHITE/BROWN,Dog Lifetime Spayed Female,2099,2005-08-08


### Removing a column

There are two ways. You could do what we did above, and just leave a column out (for a dataframe with columns "a", "b", and "c":

```Python
df = df[["a", "b"]]
```

But most dataframes have more than three columns, probably, so instead we use `.drop()`:

```Python
df = df.drop("c", axis = 1) #axis tells it you're doing a column

# OR

df = df.drop(columns=["c"])
```

In [19]:
licenses2 = licenses2.drop(columns=["ExpYear"])
licenses2 = licenses2.drop("ValidDate", axis = 1)
licenses2.head()

,OwnerZip,DogName,Breed,Color,LicenseType
0,15137,BAILEY,POODLE MIN,WHITE/BLACK/GREY,Dog Lifetime Neutered Male
1,15045,SPENCER,AM PIT BULL TERRIER,BRINDLE,Dog Lifetime Neutered Male
2,15110,CLEO,DALMATIAN,SPOTTED,Dog Lifetime Spayed Female
3,15236,HARLEY,COCKER SPANIEL,BUFF,Dog Senior Lifetime Neutered Male
4,15216,ANGEL,CHIHUAHUA,WHITE/BROWN,Dog Lifetime Spayed Female


### Adding columns

You may need to create a single new column, often based on a calculation you've done to another column. 

Weirdly enough, you can just add a column by indexing it with the name you want it to have:
    
```Python
df["new_column"] = whatever_pandas_series_or_list_you_want
```

We do have a method for it, too, though:

```Python
# one of the few Pandas operations to happen in place by default
df.insert(position, column_name, series_or_list_of_values, True/False)
```

In [21]:
# Make true/false for whether a dog has been spayed or neutered
neutered = licenses.LicenseType.str.contains("Neutered")
spayed = licenses.LicenseType.str.contains("Spayed")
# weirdly, we have to use bitwise operators to do element-by-element
# comparisons of pandas series
fixed = neutered | spayed

# add the column
licenses.insert(1, "Fixed", fixed, True)
licenses.head()

,LicenseType,Fixed,Fixed,Breed,Color,DogName,OwnerZip,ExpYear,ValidDate
0,Dog Lifetime Neutered Male,True,True,POODLE MIN,WHITE/BLACK/GREY,BAILEY,15137,2099,2005-08-08
1,Dog Lifetime Neutered Male,True,True,AM PIT BULL TERRIER,BRINDLE,SPENCER,15045,2099,2006-03-07
2,Dog Lifetime Spayed Female,True,True,DALMATIAN,SPOTTED,CLEO,15110,2099,2005-08-08
3,Dog Senior Lifetime Neutered Male,True,True,COCKER SPANIEL,BUFF,HARLEY,15236,2099,2005-08-08
4,Dog Lifetime Spayed Female,True,True,CHIHUAHUA,WHITE/BROWN,ANGEL,15216,2099,2005-08-08


It's also worth pointing out that you can "broadcast" a scalar value down a column, too.

In [22]:
licenses["IsGood"] = "Yes"
licenses.head()

,LicenseType,Fixed,Fixed,Breed,Color,DogName,OwnerZip,ExpYear,ValidDate,IsGood
0,Dog Lifetime Neutered Male,True,True,POODLE MIN,WHITE/BLACK/GREY,BAILEY,15137,2099,2005-08-08,Yes
1,Dog Lifetime Neutered Male,True,True,AM PIT BULL TERRIER,BRINDLE,SPENCER,15045,2099,2006-03-07,Yes
2,Dog Lifetime Spayed Female,True,True,DALMATIAN,SPOTTED,CLEO,15110,2099,2005-08-08,Yes
3,Dog Senior Lifetime Neutered Male,True,True,COCKER SPANIEL,BUFF,HARLEY,15236,2099,2005-08-08,Yes
4,Dog Lifetime Spayed Female,True,True,CHIHUAHUA,WHITE/BROWN,ANGEL,15216,2099,2005-08-08,Yes


### Etc.

In [29]:
dogs_by_zip = licenses.groupby("OwnerZip", as_index=False)["DogName"].count()
dogs_by_zip = dogs_by_zip.rename(columns = {"DogName" : "Count"})
dogs_by_zip.describe()
# average zip code in Allegheny County has 244 dog licenses
# median is 54
# max is 1937

ValueError: cannot reindex from a duplicate axis

In [30]:
most_dogs_by_zip = dogs_by_zip[dogs_by_zip > 200]
most_dogs_by_zip.head()

OwnerZip
15017     381
15024     347
15025     749
15037     797
15044    1105
Name: DogName, dtype: int64